In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport

matplotlib.use('Qt5Agg')

QSocketNotifier: Can only be used with threads started with QThread


In [2]:
ROOT_PATH = './../'
ENERGY_DATA_PATH = 'datasets/processed/consumption_solar_merged.csv'
REPORT_PATH = 'reports/energy_data_report.html'

In [3]:
df = pd.read_csv(ROOT_PATH + ENERGY_DATA_PATH)
profile = ProfileReport(df, title='Energy Consumption Data Profiling Report')
profile.to_file(ROOT_PATH + REPORT_PATH)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
df.head()

,Site Name,Day,Hour,Total Energy(kWh),Datetime
0,site1,1,0,4.48,2023-01-01 00:00:00
1,site1,1,1,4.30,2023-01-01 01:00:00
2,site1,1,2,4.17,2023-01-01 02:00:00
3,site1,1,3,4.15,2023-01-01 03:00:00
4,site1,1,4,4.32,2023-01-01 04:00:00


In [5]:
df.describe()

,Day,Hour,Total Energy(kWh)
count,14400.000000,14400.000000,14400.000000
mean,30.500000,11.500000,5.682772
std,17.318704,6.922427,2.704091
min,1.000000,0.000000,0.600000
25%,15.750000,5.750000,4.060000
50%,30.500000,11.500000,4.450000
75%,45.250000,17.250000,8.570000
max,60.000000,23.000000,11.670000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14400 entries, 0 to 14399
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Site Name          14400 non-null  object 
 1   Day                14400 non-null  int64  
 2   Hour               14400 non-null  int64  
 3   Total Energy(kWh)  14400 non-null  float64
 4   Datetime           14400 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 562.6+ KB


In [9]:
# Pivot the dataframe to get hours as columns and days as rows
pivot_df = df.pivot_table(index='Day', columns='Hour', values='Total Energy(kWh)', aggfunc='mean')

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(pivot_df, cmap='viridis', annot=False, fmt=".2f")
plt.title('Heatmap of Energy Consumption per Hour')
plt.xlabel('Hour')
plt.ylabel('Day')
plt.show()

# Create a pivot table for each site
for site in df['Site Name'].unique():
    site_df = df[df['Site Name'] == site]
    pivot_site_df = site_df.pivot_table(index='Day', columns='Hour', values='Total Energy(kWh)', aggfunc='mean')
    
    # Plot the heatmap for each site
    plt.figure(figsize=(12, 8))
    sns.heatmap(pivot_site_df, cmap='viridis', annot=False, fmt=".2f")
    plt.title(f'Heatmap of Energy Consumption per Hour for {site}')
    plt.xlabel('Hour')
    plt.ylabel('Day')
    plt.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [11]:
# Analyze differences between sites to determine modeling strategy
# First, let's check how many unique sites we have
sites = df['Site Name'].unique()
print(f"Number of sites: {len(sites)}")

# Compare basic statistics across sites
site_stats = df.groupby('Site Name')['Total Energy(kWh)'].describe()
print("\nBasic statistics by site:")
print(site_stats)

# Check for pattern differences between sites
plt.figure(figsize=(12, 8))
for site in sites:
    site_data = df[df['Site Name'] == site].groupby('Hour')['Total Energy(kWh)'].mean()
    plt.plot(site_data.index, site_data.values, label=site)
plt.title('Average Energy Consumption by Hour for Each Site')
plt.xlabel('Hour of Day')
plt.ylabel('Average Energy Consumption (kWh)')
plt.legend()
plt.grid(True)
plt.show()

# Recommendations
print("\nModeling Strategy Recommendations:")
print("1. Single Model Approach:")
print("   - Pros: More data for training, simpler to maintain")
print("   - Cons: May miss site-specific patterns")
print("   - Best when: Sites show similar consumption patterns")
print("\n2. Multiple Models Approach (one per site):")
print("   - Pros: Can capture site-specific patterns")
print("   - Cons: Requires more maintenance, may have less data per model")
print("   - Best when: Sites show distinct consumption patterns")
print("\nFor 15-minute predictions: Current data appears hourly. You'll need to:")
print("1. Check if 15-minute data is available")
print("2. If not, consider interpolation or obtaining more granular data")

Number of sites: 10

Basic statistics by site:
            count      mean       std   min     25%    50%      75%    max
Site Name                                                                 
site1      1440.0  4.547507  0.542484  1.91  4.3800  4.650   4.8600   5.59
site10     1440.0  4.080056  0.230967  2.48  3.9700  4.060   4.2000   6.98
site2      1440.0  9.655854  0.777470  5.54  9.1475  9.780  10.2000  11.67
site3      1440.0  5.879653  2.656415  1.31  2.7150  7.245   8.1725   9.31
site4      1440.0  9.557389  0.685180  4.88  9.0800  9.580  10.0200  11.48
site5      1440.0  8.324674  1.479104  2.02  8.2375  8.820   9.0600  10.31
site6      1440.0  4.244549  0.460316  0.60  4.1700  4.260   4.4400   4.90
site7      1440.0  4.245285  0.587350  0.79  4.1200  4.390   4.4500   5.30
site8      1440.0  2.136347  1.063300  1.31  1.7400  1.970   2.1400   8.13
site9      1440.0  4.156403  0.235679  1.53  4.0400  4.200   4.2300   4.68


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()



Modeling Strategy Recommendations:
1. Single Model Approach:
   - Pros: More data for training, simpler to maintain
   - Cons: May miss site-specific patterns
   - Best when: Sites show similar consumption patterns

2. Multiple Models Approach (one per site):
   - Pros: Can capture site-specific patterns
   - Cons: Requires more maintenance, may have less data per model
   - Best when: Sites show distinct consumption patterns

For 15-minute predictions: Current data appears hourly. You'll need to:
1. Check if 15-minute data is available
2. If not, consider interpolation or obtaining more granular data


Patterns wers shown by sites are not similar:
* site1, 6, 7, 9, 10 show us approximatly same pattern
* site2 and 4 same things
* site3 different pattern
* site5  different pattern

Based on the analysis, it seems that a one-model-fits-all approach would not capture the nuances of energy consumption across the different sites. Sites 1, 6, 7, 9, and 10 demonstrate a similar pattern, which suggests that they could be grouped for modeling. In contrast, sites 2 and 4 share their own distinct behavior, while sites 3 and 5 each display unique patterns. Therefore, it would be advisable to tailor your modeling strategy by segmenting the sites into these groups rather than using a single model for all.


In [12]:
# Define the site groups
group1_sites = ['site1', 'site6', 'site7', 'site9', 'site10']
group2_sites = ['site2', 'site4']
group3_sites = ['site3']
group4_sites = ['site5']

# Create dataframes for each group
group1_df = df[df['Site Name'].isin(group1_sites)]
group2_df = df[df['Site Name'].isin(group2_sites)]
group3_df = df[df['Site Name'].isin(group3_sites)]
group4_df = df[df['Site Name'].isin(group4_sites)]

# Quick summary of each group
print("Group 1 (sites: {}) has shape: {}".format(group1_sites, group1_df.shape))
print("Group 2 (sites: {}) has shape: {}".format(group2_sites, group2_df.shape))
print("Group 3 (sites: {}) has shape: {}".format(group3_sites, group3_df.shape))
print("Group 4 (sites: {}) has shape: {}".format(group4_sites, group4_df.shape))

Group 1 (sites: ['site1', 'site6', 'site7', 'site9', 'site10']) has shape: (7200, 5)
Group 2 (sites: ['site2', 'site4']) has shape: (2880, 5)
Group 3 (sites: ['site3']) has shape: (1440, 5)
Group 4 (sites: ['site5']) has shape: (1440, 5)


In [14]:
group4_df.head()

,Site Name,Day,Hour,Total Energy(kWh),Datetime
7200,site5,1,0,8.30,2023-01-01 00:00:00
7201,site5,1,1,7.33,2023-01-01 01:00:00
7202,site5,1,2,4.47,2023-01-01 02:00:00
7203,site5,1,3,3.10,2023-01-01 03:00:00
7204,site5,1,4,6.74,2023-01-01 04:00:00
